# Feature Engineering – Cumulative Dataframe

The purpose of this notebook is to create the cumulative dataframe. As mentioned in our presentation, this dataframe calculates the average performance of each team across all games up until its current point in the season, retaining more data, providing comprehensive team form. We will be using an iterative approach to loop over every row in the dataframe and accordingly calculate the new values for every column. A hashmap will be leveraged to maintain each team's statistics. 


In [ ]:
import pandas as pd
from ..helper_functions import get_master_df, drop_extraneous_col, save_df

### Begin Feature Engineering with Master DF

In [ ]:
df = get_master_df()
drop_extraneous_col(df)

### Team Encoding Object to Create Hashmap

In [5]:
team_encoding = { 
    # ATLANTIC
    "TOR": 1,
    "BOS": 2,
    "NYK": 3, 
    "BRK": 4,
    "PHI": 5,

    # CENTRAL
    "CLE": 6,
    "IND": 7,
    "DET": 8,
    "CHI": 9,
    "MIL": 10,

    # SOUTHEAST
    "MIA": 11,
    "ATL": 12,
    "CHO": 13,
    "WAS": 14,
    "ORL": 15,

    # NORTHWEST
    "OKC": 16,
    "POR": 17,
    "UTA": 18,
    "DEN": 19,
    "MIN": 20,

    # PACIFIC
    "GSW": 21, 
    "LAC": 22,
    "SAC": 23,
    "PHO": 24,
    "LAL": 25,

    # SOUTH WEST
    "SAS": 26,
    "DAL": 27,
    "MEM": 28,
    "HOU": 29,
    "NOP": 30
}

### Variables Required while Constructing New Columns

In [6]:
unique_stats = [col.split("_")[0] + "_cumulative" for col in df.columns if "_team0" in col and "restDays" not in col]
stats = [col.split("_")[0] for col in df.columns if "_team0" in col and "restDays" not in col]
teams = team_encoding.keys()

### Leveraging Hashmaps

While we iterate through each row in the dataframe, we will be using a hashmap to keep track of each team's statistics. It will take on a nested hashmap structure like so:

```
{
    TOR: {
        games_played = 10
        pts = 569
        3p = 89
        ...
        ...
    }
    BOS: {
        games_played = 10
        pts = 569
        3p = 89
        ...
        ...
    }
}
```

We will accumulate each team's total for each statistic. To calculate the average, we will divide the total by the current number of games played. At the beginning of each season, these values will be reset.

In [7]:
def initialize_cumulative_average():
    cumulative_averages = {}
    for team in teams:
        team_cumulative_average = {}
        for stat in unique_stats:
            team_cumulative_average[stat] = 0
        cumulative_averages[team] = team_cumulative_average
        cumulative_averages[team]["games_played"] = 0
    
    return cumulative_averages

### Initialize New Columns

In [8]:
for stat in unique_stats:
    df[f'{stat}_team0'] = 0.0
    df[f'{stat}_team1'] = 0.0

### Begin filling New Columns with Data

In [9]:
cumulative_averages = initialize_cumulative_average()
current_season = None

first_game_indices = set()
    
for index, row in df.iterrows():
    team0 = row['team0']
    team1 = row['team1']
    game_season = row['season']

    if game_season != current_season:
        cumulative_averages = initialize_cumulative_average()
        current_season = game_season

    games_played_team0 = cumulative_averages[team0]["games_played"]
    games_played_team1 = cumulative_averages[team1]["games_played"]

    if games_played_team0 == 0:
        first_game_indices.add(index)
    
    if games_played_team1 == 0:
        first_game_indices.add(index)

    if games_played_team0 > 0:
        for stat in unique_stats:
            prev_avg_team0 = cumulative_averages[team0][stat] / games_played_team0
            df.at[index, f"{stat}_team0"] = prev_avg_team0
    
    if games_played_team1 > 0:
        for stat in unique_stats:
            prev_avg_team1 = cumulative_averages[team1][stat] / games_played_team1
            df.at[index, f"{stat}_team1"] = prev_avg_team1
    
    for stat in stats:
        cumulative_averages[team0][f"{stat}_cumulative"] += row[f"{stat}_team0"]
        cumulative_averages[team1][f"{stat}_cumulative"] += row[f"{stat}_team1"]

    cumulative_averages[team0]['games_played'] += 1
    cumulative_averages[team1]['games_played'] += 1

    if index % 100 == 0:
         print(f"{index} / {len(df)}")

0 / 8468
100 / 8468
200 / 8468
300 / 8468
400 / 8468
500 / 8468
600 / 8468
700 / 8468
800 / 8468
900 / 8468
1000 / 8468
1100 / 8468
1200 / 8468
1300 / 8468
1400 / 8468
1500 / 8468
1600 / 8468
1700 / 8468
1800 / 8468
1900 / 8468
2000 / 8468
2100 / 8468
2200 / 8468
2300 / 8468
2400 / 8468
2500 / 8468
2600 / 8468
2700 / 8468
2800 / 8468
2900 / 8468
3000 / 8468
3100 / 8468
3200 / 8468
3300 / 8468
3400 / 8468
3500 / 8468
3600 / 8468
3700 / 8468
3800 / 8468
3900 / 8468
4000 / 8468
4100 / 8468
4200 / 8468
4300 / 8468
4400 / 8468
4500 / 8468
4600 / 8468
4700 / 8468
4800 / 8468
4900 / 8468
5000 / 8468
5100 / 8468
5200 / 8468
5300 / 8468
5400 / 8468
5500 / 8468
5600 / 8468
5700 / 8468
5800 / 8468
5900 / 8468
6000 / 8468
6100 / 8468
6200 / 8468
6300 / 8468
6400 / 8468
6500 / 8468
6600 / 8468
6700 / 8468
6800 / 8468
6900 / 8468
7000 / 8468
7100 / 8468
7200 / 8468
7300 / 8468
7400 / 8468
7500 / 8468
7600 / 8468
7700 / 8468
7800 / 8468
7900 / 8468
8000 / 8468
8100 / 8468
8200 / 8468
8300 / 8468
8400

### Dropping Rows

For each team's first game of each season, we need to drop its row. This is because there is no data we can use to predict the outcome of the game. Although we could use previous season stats, teams generally change quite a bit between each season. As a result, we made the decision to simply drop these rows.

In [10]:
first_game_indices
df = df.drop(first_game_indices)

### Add Back Columns

Certain columns listed in 'exceptions' cannot be averaged throughout the season. As a result, they are added back into the dataframe once the feature engineering is completed.

In [12]:
exceptions = ['team1', 'winner', 'season', 'date', 'team0', 'team0_encoded', 'team1_encoded', 'restDays_team0', 'restDays_team1']

cols_to_keep = [col for col in df.columns if "cumulative" in col or col in exceptions]

df_filtered = df[cols_to_keep]

### Determine Team 1 Winner

As mentioned in our presentation, each dataframe possesses a column relating to team1_winner. This will have a value of 1 if team1 wins and a value of 0 if team1 loses.

In [13]:
df['team1_winner'] = df.apply(lambda row: 1 if row['winner'] == row['team1'] else 0, axis=1)
df_filtered['team1_winner'] = df_filtered.apply(lambda row: 1 if row['winner'] == row['team1'] else 0, axis=1)

/var/folders/rd/xg_1cl914sb4d864cq60_8zr0000gn/T/ipykernel_90703/584802485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['team1_winner'] = df_filtered.apply(lambda row: 1 if row['winner'] == row['team1'] else 0, axis=1)


In [14]:
df_filtered.reset_index(drop=True, inplace=True)

In [17]:
save_df(df_filtered, "cumulative_averages.csv")